In [1]:
import pyspark
import os

my_dir = "./BooksDataset"

# False if you don't have the rdd saved, True otherwise
readFromSavedFile = False

#List of Authors
authors = ['Arthur Conan Doyle', 'Charles Dickens', 'Daniel Defoe', 'Edith Wharton', 'Jane Austen', 'Joseph Conrad', 'Lewis Carroll', 'Louisa May Alcott', 'Voltaire', 'William Shakespeare']

#Number of Authors
n_authors = len(authors)


In [2]:
sc = pyspark.SparkContext(appName="BigDataProject")

sc

#sc.stop()

<SparkContext master=local[*] appName=BigDataProject>

In [5]:
#Save/read all books' name of every author

if not readFromSavedFile:
    for i in range(n_authors):
        book = []
        mydiraut = my_dir+"/"+authors[i]
        print(f"Getting books' name for author: {authors[i]}")
        
        for file in os.listdir(mydiraut):
            d = os.path.join(mydiraut, file)
            book.append(file)
            
        print(f"Start - Saving RDD of all books' name of author: {authors[i]}")
        sc.parallelize(book).saveAsTextFile("./RDD/BooksName/"+authors[i])
        print(f"End - Saving RDD of all books' name of author: {authors[i]}\n")
    

print("Reading all books' name of every author from saved RDD")
books = []

for i in range(n_authors):
    books.append(sc.textFile("./RDD/BooksName/"+authors[i]+"/*").collect()) 
    
print(books)



Getting books' name for author: Arthur Conan Doyle
Start - Saving RDD of all books' name of author: Arthur Conan Doyle
End - Saving RDD of all books' name of author: Arthur Conan Doyle

Getting books' name for author: Charles Dickens
Start - Saving RDD of all books' name of author: Charles Dickens
End - Saving RDD of all books' name of author: Charles Dickens

Getting books' name for author: Daniel Defoe
Start - Saving RDD of all books' name of author: Daniel Defoe
End - Saving RDD of all books' name of author: Daniel Defoe

Getting books' name for author: Edith Wharton
Start - Saving RDD of all books' name of author: Edith Wharton
End - Saving RDD of all books' name of author: Edith Wharton

Getting books' name for author: Jane Austen
Start - Saving RDD of all books' name of author: Jane Austen
End - Saving RDD of all books' name of author: Jane Austen

Getting books' name for author: Joseph Conrad
Start - Saving RDD of all books' name of author: Joseph Conrad
End - Saving RDD of all 

In [14]:
#Function to clear the text given in input

def clearText(file_path):
    return sc.textFile(file_path).filter(bool)  \
                .map(lambda w: w.replace("."," . ")) \
                .map(lambda w: w.replace(","," , ")) \
                .map(lambda w: w.replace(";"," ; ")) \
                .map(lambda w: w.replace("!"," ! ")) \
                .map(lambda w: w.replace("?"," ? ")) \
                .map(lambda w: w.replace('"',' " ')) \
                .map(lambda w: w.replace("-"," - ")) \
                .map(lambda w: w.replace("_"," _ ")) \
                .map(lambda w: w.replace("{"," { ")) \
                .map(lambda w: w.replace("}"," } ")) \
                .map(lambda w: w.replace("["," [ ")) \
                .map(lambda w: w.replace("]"," ] ")) \
                .map(lambda w: w.replace("("," ( ")) \
                .map(lambda w: w.replace(")"," ) "))
    

In [11]:
#WordCount returning a list of words
#.filter(lambda w: w not in common_words) \

def wordCount(file_path):
    words = clearText(file_path) \
                .flatMap(lambda line: line.split(" ")) \
                .map(lambda w: w.lower()) \
                .map(lambda w: (w, 1)) \
                .reduceByKey(lambda v1, v2: v1 + v2) \
                .map(lambda x: (x[1], x[0])) \
                .sortByKey(False)
    
    result = words.collect()

    # Calculate total words of the book given in input
    total_words = 0

    for t in result:
        total_words += t[0]
    
    # Add the total word to the list of words
    result.append((total_words, 'total_words'))
    
    return result

In [15]:
#Saving RDD all words for an authors

if not readFromSavedFile:

    for i in range(1): #n_authors
        # Obtain the path of author directory
        tmp_filename = os.path.join(my_dir, authors[i])
        print(f"Word count of {authors[i]}'s books.")
        
        for j in range(2): #len(books[i])
            # Obtain the path of books given the path of an author
            input_filename = os.path.join(tmp_filename, books[i][j])
            
            print(f"Start - Counting words of book: {books[i][j]}")
            wordsOfABook = wordCount(input_filename)
            print(f"End - Counting words of book: {books[i][j]}\n")
            
            print(f"Start - Saving words count of book: {books[i][j]}")
            sc.parallelize(wordsOfABook).saveAsTextFile("./RDD/Books/"+authors[i]+"/SingleBooks/"+books[i][j])
            print(f"End - Saving words count of book: {books[i][j]}\n")
            
        


Word count of Arthur Conan Doyle's books.
Start - Counting words of book: Arthur Conan Doyle_Beyond the City.txt
End - Counting words of book: Arthur Conan Doyle_Beyond the City.txt

Start - Saving words count of book: Arthur Conan Doyle_Beyond the City.txt
End - Saving words count of book: Arthur Conan Doyle_Beyond the City.txt

Start - Counting words of book: Arthur Conan Doyle_The Adventure of the Cardboard Box.txt
End - Counting words of book: Arthur Conan Doyle_The Adventure of the Cardboard Box.txt

Start - Saving words count of book: Arthur Conan Doyle_The Adventure of the Cardboard Box.txt
End - Saving words count of book: Arthur Conan Doyle_The Adventure of the Cardboard Box.txt



In [45]:
#Given a list of string convert it to a list of tuple
def stringToTuple(wordCount):
    result = []
    for s in wordCount:
        #Remove useless characters in the string
        res = s.strip("(").strip(")").split(", ")   
        tmp = len(res[1])
        #Take only the string inside the '' 
        result.append((int(res[0]), str(res[1][1:tmp-1])))
        
    return result

In [46]:
#Read the rdd of all books of all authors
wordsCountAuthors = [[] for _ in range(n_authors)]

for i in range(1):
    print(f"Reading books for: {authors[i]}")
    for j in range(2):
        # Getting path of book's RDD
        tmp = "./RDD/Books/"+authors[i]+"/SingleBooks/"+books[i][j]+"/*"

        print(f"Start - Reading words count of book: {books[i][j]}")
        result = sc.textFile(tmp).collect()
        # Converting list of string into list of tuple
        result = stringToTuple(result)
        print(f"End - Reading words count of book: {books[i][j]}")

        wordsCountAuthors[i].append(result)
        

Reading books for: Arthur Conan Doyle
[(8717, ''), (3075, ','), (2599, '.'), (2303, '"'), (1868, 'the'), (1234, 'and'), (1003, 'a'), (955, 'to'), (896, 'of'), (827, 'i'), (687, 'you'), (658, 'it'), (561, 'in'), (559, 'that'), (466, 'is'), (461, 'he'), (457, 'her'), (444, 'was'), (444, 'his'), (442, '-'), (370, '?'), (338, 'with'), (309, 'have'), (308, 'she'), (296, 'as'), (296, 'had'), (292, 'but'), (273, 'for'), (256, '!'), (244, 'not'), (244, 'be'), (230, 'at'), (229, 'my'), (198, 'which'), (198, 'what'), (193, 'we'), (193, 'all'), (184, 'him'), (175, 'from'), (174, 'do'), (165, 'me'), (163, 'would'), (160, 'they'), (159, 'said'), (158, 'so'), (155, 'no'), (152, 'upon'), (151, 'very'), (151, 'there'), (150, 'this'), (148, 'one'), (144, 'are'), (139, 'your'), (132, 'will'), (131, 'were'), (129, 'then'), (125, 'on'), (124, 'up'), (122, 'if'), (122, 'little'), (120, 'admiral'), (115, 'man'), (113, 'their'), (110, 'clara'), (109, 'out'), (107, 'oh'), (103, 'two'), (103, 'when'), (101, 's

# Method without score NO ML

In [ ]:
#To find same words in top100 words of all authors
same_words = []

#take first author to compare the same words of the other authors 
test = wordsCountAuthors[0][0]


#Compare same words of the first author to others
for _, word in test:
    count = 0
    for i in range(1,10):
        for _, word2 in wordsCountAuthors[i]:
            if word == word2:
                count += 1
                continue
    if count == 9:
        same_words.append(word)


print(same_words)

In [ ]:
# Words in common in top100 words of all authors
common_words = ['the', ',', '.', 'of', 'and', '', 'to', 'a', 'i', 'in', 'was', 'that', 'it', 'he', 'had', 'you', 'his', 'with', 'is', 'my', 'which', 'as', 'for', 'at', 'have', 'were', 'but', 'be', 'not', 'me', 'from', 'this', 'by', 'on', 'there', 'her', 'they', 'so', 'we', 'one', 'all', 'him', 'their', 'would', 'an', 'no', '?', 'when', 'them', 'what', 'if', 'or', 'do', 'more']

In [ ]:
#rdd.take(1)[-1]

In [48]:
training_index = [0, 1, 2, 3, 4, 5, 6]
testing_index = [7, 8, 9]

#n_authors
if not(readFromSavedFile):
    for a in range(1):
        empty_rdd = sc.emptyRDD()
        #Take only the books in training_index
        for b in range(2): #training_index:
            empty_rdd = empty_rdd.union(sc.parallelize(wordsCountAuthors[a][b]))
        
        #Word Count of all books in training_index of the author
        print(f"Word Count of all books in training_index: {authors[a]}")
        result = empty_rdd.map(lambda x: (x[1], x[0])).reduceByKey(lambda v1, v2: v1 + v2) \
                .map(lambda x: (x[1], x[0])).sortByKey(False)
        
        print(result)

        total_words = result.take(1)[0]

        #Opz 1 without total words
        #result = result[1:-1]
        #result = result.map(lambda n, w: (n/total_words, w))
        result = result.map(lambda x: (x[0]/total_words, x[1]))


        #Opz 2 with total_words 
        #result = result.map(lambda n, w: (n/total_words, w) if w != "total_words" else (n, w))
        
        
        print(type(result))
        print(result)
        
        # Only save top 100 words 
        result = sc.parallelize(result.take(200))
        result.saveAsTextFile(f"./RDD/Books/{authors[a]}/TrainingResult")    
else:
    
    
    AllWordsCountAuthors = []

    for i in range(n_authors):
        print(authors[i])
        tmp = f"./RDD/Books/{authors[i]}/TrainingResult/*"   
        print(tmp)
        result = sc.textFile(tmp).collect()
        AllWordsCountAuthors.append(result)



Word Count of all books in training_index: Arthur Conan Doyle


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\giaco\anaconda3\envs\BigData\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\giaco\anaconda3\envs\BigData\lib\site-packages\py4j\clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "c:\Users\giaco\anaconda3\envs\BigData\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [11]:
AllWordCountByAuthors_RDD = [[] for _ in range(n_authors)]
for i in range(n_authors):

    for w in AllWordsCountAuthors[i]:
        print(w)
        cleaned_string = w.replace('[', '').replace(']', '').replace('(', '').replace(')', '').replace("'", '')

        word_list = cleaned_string.split(', ')

        
        for k in range(0, len(word_list), 2):
            tuple_list = (int(word_list[k]), word_list[k + 1]) 

      
        
        AllWordCountByAuthors_RDD[i].append(tuple_list)

(22086, 'the')
(21939, ',')
(16285, '.')
(10278, 'of')
(9630, 'and')
(9243, '')
(7811, 'to')
(7490, 'a')
(6378, 'i')
(5397, 'in')
(5232, 'was')
(4601, 'that')
(4417, 'it')
(3201, 'he')
(3003, 'had')
(2952, 'you')
(2764, 'his')
(2614, 'with')
(2517, '-')
(2496, 'is')
(2495, 'my')
(2410, 'which')
(2364, 'as')
(2323, 'for')
(2211, 'at')
(2107, 'have')
(2014, 'were')
(1919, 'but')
(1767, '"')
(1738, 'be')
(1676, 'not')
(1669, 'me')
(1634, 'from')
(1624, 'upon')
(1525, 'this')
(1491, 'by')
(1469, '.”')
(1419, 'on')
(1286, 'there')
(1247, 'her')
(1220, 'they')
(1216, 'been')
(1213, 'so')
(1204, 'we')
(1156, 'one')
(1154, 'all')
(1128, 'him')
(1110, 'their')
(1073, 'said')
(1012, 'would')
(1003, 'she')
(989, 'an')
(949, 'no')
(937, 'could')
(894, '"s"')
(889, 'who')
(887, 'up')
(886, 'very')
(844, 'out')
(834, '?')
(791, 'when')
(784, 'them')
(759, 'are')
(757, 'will')
(756, 'what')
(735, 'your')
(733, '!')
(729, 'some')
(728, ',”')
(717, 'into')
(707, 'man')
(704, 'then')
(668, 'if')
(640, '

In [40]:

#Change to select the rigth author
test_author = 0

"""
tmp_filename = os.path.join("./RDD/Books", authors[test_author])
input_filename = os.path.join(tmp_filename, books[test_author][testing_index[0]])
input_filename = input_filename
lista2 = sc.textFile(input_filename).collect()
"""

lista2 = wordCountByAuthors[test_author][testing_index[0]].take(100)
print(lista2)

def count_common_words(lista1, lista2):
    count_words = 0

    for parola in lista1:
        #print(parola[1])
        if parola in lista2:
            count_words += 1

    return count_words


#print(count_common_words(AllWordCountByAuthors_RDD[0], lista2))

max_count = -1
author_index = []

for a in range(n_authors):
    c = []
    for p in AllWordCountByAuthors_RDD[a]:
        c.append(p[1])

    if count_common_words(c, lista2) == max_count:
        author_index.append(a)

    if count_common_words(c, lista2) > max_count:
        author_index = []
        author_index.append(a)
        max_count = count_common_words(c, lista2) 




for k in author_index:
    print(authors[k])

[(5449, ','), (5205, 'the'), (3629, '.'), (2644, 'and'), (2621, 'of'), (2103, 'a'), (2073, 'to'), (1861, 'i'), (1340, 'in'), (1293, 'was'), (1105, 'he'), (1073, 'that'), (1049, 'his'), (1033, 'it'), (819, 'as'), (803, 'my'), (720, 'had'), (702, 'with'), (651, 'which'), (597, 'for'), (570, 'at'), (553, 'is'), (516, 'me'), (491, 'you'), (481, 'have'), (468, 'upon'), (427, 'from'), (422, 'him'), (404, 'but'), (401, 'be'), (400, 'her'), (388, 'on'), (377, 'by'), (371, 'this'), (354, 'one'), (349, ''), (347, 'there'), (339, ',”'), (336, '-'), (324, 'were'), (309, '.”'), (296, 'not'), (285, 'all'), (281, 'an'), (273, 'so'), (269, 'out'), (264, 'been'), (263, 'when'), (262, 'said'), (256, 'we'), (255, 'she'), (254, 'up'), (252, 'some'), (244, 'man'), (235, 'into'), (235, 'who'), (230, 'would'), (217, 'their'), (216, 'they'), (215, 'no'), (210, 'or'), (204, 'could'), (192, 'down'), (187, 'are'), (180, 'what'), (178, 'over'), (176, ';'), (176, 'more'), (172, 'them'), (168, 'then'), (163, 'very'

['the', ',', '.', 'of', 'and', '', 'to', 'a', 'i', 'in', 'was', 'that', 'it', 'he', 'had', 'you', 'his', 'with', 'is', 'my', 'which', 'as', 'for', 'at', 'have', 'were', 'but', 'be', 'not', 'me', 'from', 'this', 'by', 'on', 'there', 'her', 'they', 'so', 'we', 'one', 'all', 'him', 'their', 'would', 'an', 'no', '?', 'when', 'them', 'what', 'if', 'or', 'do', 'more']


# Celle di Test

In [8]:
n_books = []

for i in range(len(books)):
    data_str = str(books[i])
    data_str = data_str.replace("[", "").replace("]", "").replace("'", "")
    data_list = data_str.split(", ")
    n_books.append(len(data_list))

print(n_books)


[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]


In [45]:
from pyspark.sql.types import StructType, StructField, StringType
import pandas as pd

In [61]:
#Creo il dataframe
training_data = []

for i in range(n_authors):
    training_data.append((authors[i], AllWordCountByAuthors_RDD[i]))


df = pd.DataFrame(training_data)
df.columns = ["Author", "Top100Words"]
print(df)




                Author                                        Top100Words
0   Arthur Conan Doyle  [(28688, the), (13417, of), (12559, and), (101...
1      Charles Dickens  [(69026, the), (50940, and), (39832, to), (379...
2         Daniel Defoe  [(24009, the), (16405, and), (14316, to), (134...
3        Edith Wharton  [(29289, the), (18053, ), (16213, of), (14380,...
4          Jane Austen  [(42721, ), (32293, the), (29439, to), (26255,...
5        Joseph Conrad  [(38777, the), (24566, of), (17697, a), (17400...
6        Lewis Carroll  [(39778, ), (11110, the), (5736, and), (5106, ...
7    Louisa May Alcott  [(28537, the), (24400, and), (16083, to), (145...
8             Voltaire  [(26520, the), (16079, ), (16067, of), (11568,...
9  William Shakespeare  [(46471, ), (7975, the), (7023, and), (5525, i...


In [62]:
text_to_predict = [(3, "Ciao"), (4,"hello"), (2,"first"), (5,"british")]

In [11]:
#[Authors][#Books]
#wordCountByAuthors[0][0]

empty_rdd = sc.emptyRDD()

for i in range(10):
    empty_rdd = empty_rdd.union(wordCountByAuthors[0][i])

#print(empty_rdd.collect())




In [12]:
#tmp = empty_rdd.flatMap().map(lambda x: (x[1], x[0])).reduceByKey(lambda v1, v2: v1 + v2) \
#                .map(lambda x: (x[1], x[0])) \
                

tmp = empty_rdd.map(lambda x: (x[1], x[0])).reduceByKey(lambda x,y: x+y)
print(tmp.collect())

[('something', 187), ('"she', 16), ('family', 40), ('lit', 38), ('business,', 23), ('quick', 47), ('yours', 23), ('led', 128), ('sometimes', 44), ('step', 42), ('opportunity', 22), ('agent', 11), ('especially', 52), ('matters', 36), ('smith', 26), ('papers,', 16), ('manager', 8), ('peace.', 6), ('strangers', 6), ('eggs.', 3), ('results', 19), ('unhappy', 13), ('truly,', 5), ('"go', 2), ('policy', 14), ('curtains.', 2), ('like,', 13), ('unendurable', 2), ('horrible', 25), ('stuck', 12), ('no,"', 2), ('clara."', 2), ('patted', 5), ('guard', 51), ('here.', 28), ('table.', 34), ('confessed', 8), ('clubs', 2), ('labors,', 1), ('ii.', 9), ('cross,', 3), ('vouched', 2), ('placard,', 1), ('diathesis"', 1), ('treatise', 2), ('unoccupied,', 1), ('abhorrence.', 1), ('meetings.', 2), ('forming', 23), ('athletic', 2), ('spare,', 1), ('bread--poor,', 1), ('phonographs,', 1), ('iv.', 5), ('it"--he', 1), ('"setebos."', 1), ('mother--to', 1), ('spire', 1), ('pace.', 1), ('clearcut,', 1), ('ideas,', 1),

In [14]:
tmp2 = tmp.map(lambda x:(x[1], x[0]))

print(tmp2.collect())

[(187, 'something'), (16, '"she'), (40, 'family'), (38, 'lit'), (23, 'business,'), (47, 'quick'), (23, 'yours'), (128, 'led'), (44, 'sometimes'), (42, 'step'), (22, 'opportunity'), (11, 'agent'), (52, 'especially'), (36, 'matters'), (26, 'smith'), (16, 'papers,'), (8, 'manager'), (6, 'peace.'), (6, 'strangers'), (3, 'eggs.'), (19, 'results'), (13, 'unhappy'), (5, 'truly,'), (2, '"go'), (14, 'policy'), (2, 'curtains.'), (13, 'like,'), (2, 'unendurable'), (25, 'horrible'), (12, 'stuck'), (2, 'no,"'), (2, 'clara."'), (5, 'patted'), (51, 'guard'), (28, 'here.'), (34, 'table.'), (8, 'confessed'), (2, 'clubs'), (1, 'labors,'), (9, 'ii.'), (3, 'cross,'), (2, 'vouched'), (1, 'placard,'), (1, 'diathesis"'), (2, 'treatise'), (1, 'unoccupied,'), (1, 'abhorrence.'), (2, 'meetings.'), (23, 'forming'), (2, 'athletic'), (1, 'spare,'), (1, 'bread--poor,'), (1, 'phonographs,'), (5, 'iv.'), (1, 'it"--he'), (1, '"setebos."'), (1, 'mother--to'), (1, 'spire'), (1, 'pace.'), (1, 'clearcut,'), (1, 'ideas,'),

In [15]:
tmp3 = tmp2.sortByKey(False)
print(tmp3)

PythonRDD[170] at RDD at PythonRDD.scala:53


In [16]:
print(tmp3.collect())

[(28688, 'the'), (13417, 'of'), (12559, 'and'), (10152, 'to'), (9959, 'a'), (8018, 'i'), (6971, 'in'), (6828, ''), (6761, 'was'), (5835, 'that'), (4637, 'it'), (4247, 'he'), (3964, 'his'), (3900, 'had'), (3396, 'with'), (3305, 'my'), (3300, 'as'), (3141, 'which'), (3108, 'is'), (2992, 'for'), (2915, 'you'), (2896, 'at'), (2654, 'have'), (2387, 'were'), (2353, 'but'), (2193, 'be'), (2164, 'upon'), (2145, 'from'), (1985, 'not'), (1956, 'by'), (1921, 'this'), (1855, 'on'), (1555, 'been'), (1504, 'her'), (1501, 'there'), (1481, 'they'), (1461, 'we'), (1388, 'one'), (1388, 'me'), (1356, 'their'), (1343, 'all'), (1331, 'she'), (1320, 'so'), (1318, 'an'), (1295, 'would'), (1176, 'could'), (1168, 'said'), (1162, 'who'), (1122, 'no'), (1087, 'very'), (1060, 'when'), (1023, 'out'), (1021, 'some'), (1013, 'him'), (1010, 'up'), (972, 'into'), (949, 'what'), (910, 'are'), (906, 'or'), (892, 'will'), (877, 'your'), (853, 'if'), (781, 'has'), (746, 'man'), (732, 'our'), (714, 'two'), (692, 'little'),

In [17]:
tmp1000 = empty_rdd.map(lambda x: (x[1], x[0])).reduceByKey(lambda v1, v2: v1 + v2) \
               .map(lambda x: (x[1], x[0])).sortByKey(False)
print(tmp1000.collect())

[(28688, 'the'), (13417, 'of'), (12559, 'and'), (10152, 'to'), (9959, 'a'), (8018, 'i'), (6971, 'in'), (6828, ''), (6761, 'was'), (5835, 'that'), (4637, 'it'), (4247, 'he'), (3964, 'his'), (3900, 'had'), (3396, 'with'), (3305, 'my'), (3300, 'as'), (3141, 'which'), (3108, 'is'), (2992, 'for'), (2915, 'you'), (2896, 'at'), (2654, 'have'), (2387, 'were'), (2353, 'but'), (2193, 'be'), (2164, 'upon'), (2145, 'from'), (1985, 'not'), (1956, 'by'), (1921, 'this'), (1855, 'on'), (1555, 'been'), (1504, 'her'), (1501, 'there'), (1481, 'they'), (1461, 'we'), (1388, 'one'), (1388, 'me'), (1356, 'their'), (1343, 'all'), (1331, 'she'), (1320, 'so'), (1318, 'an'), (1295, 'would'), (1176, 'could'), (1168, 'said'), (1162, 'who'), (1122, 'no'), (1087, 'very'), (1060, 'when'), (1023, 'out'), (1021, 'some'), (1013, 'him'), (1010, 'up'), (972, 'into'), (949, 'what'), (910, 'are'), (906, 'or'), (892, 'will'), (877, 'your'), (853, 'if'), (781, 'has'), (746, 'man'), (732, 'our'), (714, 'two'), (692, 'little'),

In [91]:
sc.stop()